# Create Anonymised Sample Dataset

This notebook creates a privacy-protected sample of the eBay sales data for portfolio sharing.

**Purpose:**
- Protect business-sensitive information
- Create reproducible sample for public GitHub repository
- Maintain data distributions for meaningful analysis


## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import hashlib
import re
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split

## 2. Configuration

In [2]:
# File paths
DATA_PATH = '../data/raw/'
OUTPUT_PATH = '../data/'

# Sampling parameters
SAMPLE_SIZE = 1000
RANDOM_STATE = 42

# Anonymisation settings
PRICE_NOISE_RANGE = (0.95, 1.05)  # ±5% price variation
DATE_OFFSET_DAYS = 30  # Max days to shift dates

# Columns to drop (based on your original analysis)
DROP_COLUMNS = ['Feedback left', 'Feedback received', 
                'Global Shipping Programme', 'Click and Collect']

## 3. Load Original Data

In [3]:
df = pd.read_csv(f'{DATA_PATH}ebay_march2023_feb2025_less-cols.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Basic info
print("\nData types:")
print(df.dtypes)

Original dataset shape: (8778, 14)
Columns: ['Sales record number', 'Order number', 'Buyer username', 'Buyer postcode', 'Item number', 'Item title', 'Sold via Promoted listings', 'Quantity', 'Sold for', 'Sale date', 'Feedback left', 'Feedback received', 'Global Shipping Programme', 'Click and Collect']

Data types:
Sales record number           float64
Order number                   object
Buyer username                 object
Buyer postcode                 object
Item number                   float64
Item title                     object
Sold via Promoted listings     object
Quantity                      float64
Sold for                       object
Sale date                      object
Feedback left                  object
Feedback received              object
Global Shipping Programme      object
Click and Collect              object
dtype: object


## 3.5 Handle eBay's Parent-Child Row Structure

eBay exports data with parent rows (order-level) followed by child rows (item-level). Parent rows have missing item details.


In [4]:
# Check for parent rows (missing item details)
print("Data structure analysis:")
print(f"Total rows: {len(df)}")
print(f"Rows with missing Item title: {df['Item title'].isna().sum()}")
print(f"Percentage of parent rows: {(df['Item title'].isna().sum() / len(df) * 100):.1f}%")

# Look at example of parent-child structure
parent_rows = df[df['Item title'].isna()]
if len(parent_rows) > 0:
    sample_order = parent_rows.iloc[0]['Order number']
    print(f"\nExample - Order {sample_order}:")
    display(df[df['Order number'] == sample_order][['Sales record number', 'Order number', 'Item title', 'Quantity', 'Sold for']])


Data structure analysis:
Total rows: 8778
Rows with missing Item title: 341
Percentage of parent rows: 3.9%

Example - Order nan:


,Sales record number,Order number,Item title,Quantity,Sold for


In [5]:
# Cell: Code - Fix eBay Parent-Child Structure
def fix_ebay_structure(df):
    """
    Fix eBay's parent-child export structure.
    Parent rows: Have order info but empty Item number/Item title
    Child rows: Have item details
    """
    # Create a copy
    df_work = df.copy()
    
    # Parent rows are those with missing Item title AND Item number
    parent_mask = df_work['Item title'].isna() & df_work['Item number'].isna()
    parent_rows = df_work[parent_mask]
    child_rows = df_work[~parent_mask]
    
    print(f"Found {len(parent_rows)} parent rows (order summaries)")
    print(f"Found {len(child_rows)} child rows (actual items)")
    
    # Check for any parent rows with missing order numbers
    missing_order = parent_rows['Order number'].isna().sum()
    if missing_order > 0:
        print(f"WARNING: {missing_order} parent rows have missing order numbers - these will be dropped")
        parent_rows = parent_rows[parent_rows['Order number'].notna()]
    
    # For each order, propagate parent row info to children if needed
    # (though in your example, child rows seem complete)
    
    # Simply return child rows since they have all needed info
    df_clean = child_rows.copy()
    
    # Verify the data
    print(f"\nCleaned dataset: {len(df_clean)} item-level records")
    print(f"Unique orders: {df_clean['Order number'].nunique()}")
    
    # Quick validation - no more missing item titles
    assert df_clean['Item title'].notna().all(), "Still have missing item titles!"
    
    return df_clean

# Apply the fix
df = fix_ebay_structure(df)

# Verify
print("\nFirst few rows of cleaned data:")
display(df.head(3))

Found 341 parent rows (order summaries)
Found 8437 child rows (actual items)

Cleaned dataset: 8437 item-level records
Unique orders: 8037

First few rows of cleaned data:


,Sales record number,Order number,Buyer username,Buyer postcode,Item number,Item title,Sold via Promoted listings,Quantity,Sold for,Sale date,Feedback left,Feedback received,Global Shipping Programme,Click and Collect
1,9405.0,08-12772-14508,uk.scooby1,l21 9LX,1.162940e+11,Dr. S. Wong's Sulfur Moisturising Soap 80g - A...,Yes,1.0,£6.99,28-Feb-25,Yes,positive,No,No
2,9404.0,03-12778-00244,hilaridalm-0,BN3 8GG,1.154920e+11,Gluta-C Glutathione & Kojic Plus Acne Control ...,No,1.0,£8.99,28-Feb-25,No,NaN,No,No
3,9403.0,11-12768-15354,raconye-82,L36LG,1.155650e+11,Kojie San Soap 100g x 3 (Large Trio Pack) - Sk...,No,1.0,£8.69,28-Feb-25,No,NaN,No,No


## 4. Extract Brand Information (for stratified sampling)

In [6]:
# Brand list
BRANDS = ["Kojie San", "Extract", "Gluta-C", "Silka", "Belo", 
          "Maxi-Peel", "Likas", "GlutaMax", "SkinWhite", "Glupa"]

In [7]:
def extract_brand(title):
    """Extract brand from item title."""
    if pd.isna(title):
        return "Other"
    
    title_lower = str(title).lower()
    for brand in BRANDS:
        if brand.lower() in title_lower:
            return brand
    return "Other"


In [8]:
# Extract brand for stratified sampling
df['Brand'] = df['Item title'].apply(extract_brand)
print("\nBrand distribution:")
print(df['Brand'].value_counts())


Brand distribution:
Brand
Kojie San    3559
Other        1358
Extract      1340
Silka         705
Gluta-C       645
Belo          300
SkinWhite     161
GlutaMax      125
Likas         111
Maxi-Peel     107
Glupa          26
Name: count, dtype: int64


## 5. Create Stratified Sample

In [9]:
# Stratified sampling to maintain brand distribution
sample_df = df.groupby('Brand', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), max(10, int(SAMPLE_SIZE * len(x) / len(df)))), 
                       random_state=RANDOM_STATE)
).reset_index(drop=True)

# Ensure we have roughly SAMPLE_SIZE rows
if len(sample_df) > SAMPLE_SIZE:
    sample_df = sample_df.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE)

print(f"\nSample size: {len(sample_df)}")
print(f"Sample brand distribution:\n{sample_df['Brand'].value_counts()}")


Sample size: 1000
Sample brand distribution:
Brand
Kojie San    421
Other        160
Extract      157
Silka         83
Gluta-C       76
Belo          35
SkinWhite     19
GlutaMax      14
Likas         13
Maxi-Peel     12
Glupa         10
Name: count, dtype: int64


/var/folders/2w/kfdqhq5j5kq8v8tvtj8qfm6w0000gn/T/ipykernel_72355/3273040689.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = df.groupby('Brand', group_keys=False).apply(


## 6. Anonimise Sensitive Data

In [10]:
# Create working copy
anon_df = sample_df.copy()

# 1. Anonymise buyer usernames
anon_df['Buyer username'] = anon_df['Buyer username'].apply(
    lambda x: 'user_' + hashlib.md5(str(x).encode()).hexdigest()[:8] 
    if pd.notna(x) else 'anonymous'
)

In [11]:
# 2. Generalise postcodes (keep only area)
anon_df['Buyer postcode'] = anon_df['Buyer postcode'].apply(
    lambda x: str(x).split()[0] if pd.notna(x) and str(x) != 'nan' else 'XX1'
)

In [12]:
# 3. Anonymise item numbers
anon_df['Item number'] = anon_df['Item number'].apply(
    lambda x: 'item_' + hashlib.md5(str(x).encode()).hexdigest()[:10]
    if pd.notna(x) else 'item_unknown'
)

In [13]:
# 4. Add noise to prices while preserving discount structure
def calculate_unit_price(sold_for, quantity):
    """Calculate original unit price from total after discount."""
    if quantity == 1:
        return sold_for
    elif quantity == 2:
        return sold_for / (2 * 0.9)
    elif quantity == 3:
        return sold_for / (3 * 0.85)
    else:  # 4+
        return sold_for / (quantity * 0.8)
    
# Extract prices and quantities
anon_df['Price_Numeric'] = anon_df['Sold for'].str.replace('£', '').astype(float)

# Calculate unit prices
anon_df['Unit_Price'] = anon_df.apply(
    lambda row: calculate_unit_price(row['Price_Numeric'], row['Quantity']), 
    axis=1
)

# Add noise to UNIT price (not total)
noise = np.random.uniform(0.95, 1.05, len(anon_df))
anon_df['Unit_Price_Noisy'] = anon_df['Unit_Price'] * noise

# Recalculate total with discount structure
def calculate_total_price(unit_price, quantity):
    """Calculate total price with quantity discounts."""
    if quantity == 1:
        return unit_price
    elif quantity == 2:
        return unit_price * 2 * 0.9
    elif quantity == 3:
        return unit_price * 3 * 0.85
    else:  # 4+
        return unit_price * quantity * 0.8

anon_df['Sold for'] = anon_df.apply(
    lambda row: f"£{calculate_total_price(row['Unit_Price_Noisy'], row['Quantity']):.2f}",
    axis=1
)
    
# Clean up temporary columns
anon_df = anon_df.drop(columns=['Price_Numeric', 'Unit_Price', 'Unit_Price_Noisy'])

In [14]:
# 5. Shift dates (maintain relative ordering)
anon_df['Sale date'] = pd.to_datetime(anon_df['Sale date'])
date_offset = np.random.randint(-DATE_OFFSET_DAYS, DATE_OFFSET_DAYS)
anon_df['Sale date'] = anon_df['Sale date'] + pd.Timedelta(days=date_offset)
anon_df['Sale date'] = anon_df['Sale date'].dt.strftime('%d-%b-%y')

print("Anonymisation complete!")

Anonymisation complete!


/var/folders/2w/kfdqhq5j5kq8v8tvtj8qfm6w0000gn/T/ipykernel_72355/3681370098.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  anon_df['Sale date'] = pd.to_datetime(anon_df['Sale date'])


In [15]:
# Add after calculating unit prices
def calculate_discount_percentage(quantity):
    """Return discount percentage based on quantity."""
    if quantity == 1:
        return 0
    elif quantity == 2:
        return 10
    elif quantity == 3:
        return 15
    else:  # 4+
        return 20

anon_df['Discount_Percentage'] = anon_df['Quantity'].apply(calculate_discount_percentage)

In [16]:
categories = {
    'Face Cream': r'face.*cream|Silka Papaya Day|Gluta-C Facial Face Night',
    'Face Wash': r'facial|wash|face',
    'Lotion': r'lotion',
    'Soap': r'soap',
    'Body Washes and Scrubs': r'body wash|scrub|body scrub',
    'Toner/Cleanser': r'toner|cleanser|Maxi-Peel Zero',
    'Serum': r'serum',
    'Shampoo': r'shampoo',
    'Conditioner': r'conditioner',
    'Powder': r'powder',
    'Sensitive Area Products': r'underarm|bikini|gel|roll on|roll-on|deodorant|feminine wash',
    'Cologne': r'cologne'
}

def extract_product_category(title):
    """Extract general product category from title."""
    title_lower = str(title).lower()
    
    # Your existing logic
    for category, pattern in categories.items():
        if re.search(pattern, title_lower):
            return category
    return "Other"

def extract_product_size(title):
    """Extract size information if available."""
    title_lower = str(title).lower()
    
    # Common size patterns
    size_patterns = {
        r'(\d+)\s*g\b': 'grams',
        r'(\d+)\s*ml\b': 'ml',
        r'(\d+)\s*oz\b': 'oz',
        r'(\d+)\s*pcs?\b': 'pieces'
    }
    
    for pattern, unit in size_patterns.items():
        match = re.search(pattern, title_lower)
        if match:
            return f"{match.group(1)}{unit}"
    return "standard"

# Add to your anonymization
anon_df['Product_Category'] = anon_df['Item title'].apply(extract_product_category)
anon_df['Product_Size'] = anon_df['Item title'].apply(extract_product_size)
anon_df['Discount_Percentage'] = anon_df['Quantity'].apply(calculate_discount_percentage)

In [ ]:
# Track Extract bundles as multiple quantities of the same item
# This is a specific case where bundles affect quantity, not pack count.
# For example, "Extract Papaya Calamansi Soap 6 x 125g BUNDLE" means you must buy 6 soaps, not that each pack contains 6 soaps.

def extract_product_details(title):
    """Extract product details - bundles affect quantity, not pack count."""
    if pd.isna(title):
        return {
            'Unit_Size': 'unknown',
            'Pack_Count': 1,
            'Is_Multipack': False,
            'Is_Bundle': False,
            'Bundle_Qty': 1
        }
    
    title_str = str(title)
    title_lower = title_str.lower()
    
    # Check if it's a bundle
    is_bundle = 'bundle' in title_lower
    
    # Initialize defaults
    unit_size = 'unknown'
    pack_count = 1  # Default pack count
    bundle_qty = 1  # How many items in the bundle
    
    # For bundles, extract the quantity (e.g., "6 x 125g BUNDLE" -> bundle_qty = 6)
    if is_bundle:
        # Pattern for bundles: "6 x 125g"
        bundle_pattern = r'(\d+)\s*x\s*(\d+)\s*(g|ml|oz)'
        match = re.search(bundle_pattern, title_lower)
        if match:
            bundle_qty = int(match.group(1))  # This is the bundle quantity
            unit_size = f"{match.group(2)}{match.group(3)}"
    
    # For multipacks (not bundles), look for "size x count" pattern
    elif 'x' in title_lower and not is_bundle:
        # Pattern: "135g x 2" (multipack)
        multipack_pattern = r'(\d+)\s*(g|ml|oz)\s*x\s*(\d+)'
        match = re.search(multipack_pattern, title_lower)
        if match:
            unit_size = f"{match.group(1)}{match.group(2)}"
            pack_count = int(match.group(3))  # This is pack count for multipacks
    
    # Single item pattern
    if unit_size == 'unknown':
        single_pattern = r'(\d+)\s*(g|ml|oz)\b'
        match = re.search(single_pattern, title_lower)
        if match:
            unit_size = f"{match.group(1)}{match.group(2)}"
    
    is_multipack = (pack_count > 1) and not is_bundle
    
    return {
        'Unit_Size': unit_size,
        'Pack_Count': pack_count,
        'Is_Multipack': is_multipack,
        'Is_Bundle': is_bundle,
        'Bundle_Qty': bundle_qty  # How many items must be bought as a bundle
    }

# Test this understanding
test_cases = [
    "Extract Papaya Calamansi Soap 6 x 125g BUNDLE",  # Bundle: must buy 6
    "Kojie San Soap 135g x 2",  # Multipack: 1 pack contains 2 soaps
    "Some Bundle Deal 12 x 50g BUNDLE",  # Bundle: must buy 12
]

print("Testing bundle vs multipack logic:")
for test in test_cases:
    result = extract_product_details(test)
    print(f"\n{test}")
    print(f"  Is Bundle: {result['Is_Bundle']}, Bundle Qty: {result['Bundle_Qty']}")
    print(f"  Is Multipack: {result['Is_Multipack']}, Pack Count: {result['Pack_Count']}")

# Now the important part - adjust Quantity for bundles!
print("\nAdjusting quantities for bundles...")

# Apply extraction
product_details = anon_df['Item title'].apply(extract_product_details)
anon_df['Unit_Size'] = product_details.apply(lambda x: x['Unit_Size'])
anon_df['Pack_Count'] = product_details.apply(lambda x: x['Pack_Count'])
anon_df['Is_Multipack'] = product_details.apply(lambda x: x['Is_Multipack'])
anon_df['Is_Bundle'] = product_details.apply(lambda x: x['Is_Bundle'])
anon_df['Bundle_Qty'] = product_details.apply(lambda x: x['Bundle_Qty'])

# For bundles, the quantity should be the bundle quantity
# Save original quantity first
anon_df['Original_Quantity'] = anon_df['Quantity']

# Update quantity for bundles
anon_df.loc[anon_df['Is_Bundle'], 'Quantity'] = anon_df.loc[anon_df['Is_Bundle'], 'Bundle_Qty']

# Now calculate total units correctly
# For bundles: Bundle_Qty * Original_Quantity (if they bought multiple bundles)
# For multipacks: Pack_Count * Quantity
# For singles: Quantity
anon_df['Total_Units'] = anon_df.apply(
    lambda row: row['Bundle_Qty'] * row['Original_Quantity'] if row['Is_Bundle'] 
    else row['Pack_Count'] * row['Quantity'], 
    axis=1
)

Testing bundle vs multipack logic:

Extract Papaya Calamansi Soap 6 x 125g BUNDLE
  Is Bundle: True, Bundle Qty: 6
  Is Multipack: False, Pack Count: 1

Kojie San Soap 135g x 2
  Is Bundle: False, Bundle Qty: 1
  Is Multipack: True, Pack Count: 2

Some Bundle Deal 12 x 50g BUNDLE
  Is Bundle: True, Bundle Qty: 12
  Is Multipack: False, Pack Count: 1

Adjusting quantities for bundles...


In [19]:
# Drop unnecessary columns
columns_to_drop = ['Feedback left', 'Feedback received', 
                   'Global Shipping Programme', 'Click and Collect']
final_df = anon_df.drop(columns=columns_to_drop, errors='ignore')

# Save the enhanced sample dataset
final_df.to_csv('../data/sample_data.csv', index=False)
print(f"Enhanced sample dataset saved with {len(final_df)} rows and {len(final_df.columns)} columns")
print(f"Columns: {final_df.columns.tolist()}")

# Quick validation
print("\nSample of anonymised data:")
print(final_df[['Buyer username', 'Item title', 'Quantity', 'Total_Units', 'Sold for', 'Discount_Percentage']].head())

Enhanced sample dataset saved with 1000 rows and 21 columns
Columns: ['Sales record number', 'Order number', 'Buyer username', 'Buyer postcode', 'Item number', 'Item title', 'Sold via Promoted listings', 'Quantity', 'Sold for', 'Sale date', 'Brand', 'Discount_Percentage', 'Product_Category', 'Product_Size', 'Unit_Size', 'Pack_Count', 'Is_Multipack', 'Is_Bundle', 'Bundle_Qty', 'Original_Quantity', 'Total_Units']

Sample of anonymised data:
    Buyer username                                         Item title  \
521  user_5498d0eb  Kojie San Soap 65g x 3 (Triple Pack) - Skin Br...   
941  user_c870a882  Silka Papaya Lotion SPF6 200ml - Skin Lighteni...   
741  user_39745765  Dr. Kaufmann Sulfur Soap 80g - Dual Skin Prote...   
980  user_f38d6a52  Silka Papaya Lotion SPF6 200ml - Skin Lighteni...   
411  user_9608f32f  Kojie San Body Lotion 200g - Lightening & Brig...   

     Quantity  Total_Units Sold for  Discount_Percentage  
521       1.0          3.0    £6.80                    0  
